In [ ]:
# Load and read PDF content
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader("pdf/DCPR_2034_13-09-2024.pdf")
docs = loader.load()
print("Number of pages in the PDF:", len(docs))

# Extract text from each page for embedding
doc_texts = [page.page_content for page in docs]
#print(" Document  texts",doc_texts)


In [2]:
from milvus_model.hybrid import BGEM3EmbeddingFunction
import torch
device = "mps" if torch.backends.mps.is_available() else "cpu"
ef = BGEM3EmbeddingFunction(use_fp16=False, device=device)
dense_dim = ef.dim["dense"]

# Generate embeddings using BGE-M3 model
docs_embeddings = ef(doc_texts)

/Users/krishna/Documents/PROJECTS/pythonproject/Github/chatbotapp/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 206615.96it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [3]:
dense_dim

1024

In [1]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [2]:
# Connect to Milvus given URI
connections.connect(uri="./milvus.db")

# Specify the data schema for the new Collection
fields = [
    # Use auto generated id as primary key
    FieldSchema(
        name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=True, max_length=100
    ),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2048),
    # Milvus now supports both sparse and dense vectors,
    # we can store each in a separate field to conduct hybrid search on both vectors
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=dense_dim),
]
schema = CollectionSchema(fields)

NameError: name 'dense_dim' is not defined

In [7]:
# Create collection (drop the old one if exists)
col_name = "hybrid_demo"
if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Strong")

# To make vector search efficient, we need to create indices for the vector fields
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
col.create_index("dense_vector", dense_index)
col.load()

In [8]:
# For efficiency, we insert 50 records in each small batch
for i in range(0, len(doc_texts), 50):
    # Extract the raw text from Document objects
    batched_texts = [doc.page_content for doc in docs[i : i + 50]]  # Extract the text content
    batched_sparse_vectors = docs_embeddings["sparse"][i : i + 50]
    batched_dense_vectors = docs_embeddings["dense"][i : i + 50]

    # Prepare the batched entities for insertion
    batched_entities = [
        batched_texts,          # Text content
        batched_sparse_vectors, # Sparse vector embeddings
        batched_dense_vectors,  # Dense vector embeddings
    ]

    # Insert the batch into Milvus
    col.insert(batched_entities)

print("Number of entities inserted:", col.num_entities)

Number of entities inserted: 32


In [24]:
# Generate embeddings for the query
query = "What is the minimum requirement of FSI for building above 70m"
query_embeddings = ef([query])
print(query_embeddings)

{'dense': [array([-0.02032133, -0.00229368,  0.00926588, ..., -0.03365859,
       -0.04196751,  0.02864992], dtype=float32)], 'sparse': <Compressed Sparse Row sparse array of dtype 'float64'
	with 14 stored elements and shape (1, 250002)>}


In [25]:
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)

def dense_search(col, query_dense_embedding, limit=5):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_dense_embedding],
        anns_field="dense_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

In [26]:
def sparse_search(col, query_sparse_embedding, limit=5):
    search_params = {
        "metric_type": "IP", "params": {}, }
    res = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]

In [27]:
def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=5,
):
    dense_search_params = {"metric_type": "IP", "params": {}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )
    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["text"]
    )[0]
    return [hit.get("text") for hit in res]

In [28]:
dense_results = dense_search(col, query_embeddings["dense"][0])
sparse_results = sparse_search(col, query_embeddings["sparse"][[0]])
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense"][0],
    query_embeddings["sparse"][[0]],
    sparse_weight=0.7,
    dense_weight=1.0,
)

In [29]:
print(dense_results)

['1\n1\nREGULATION 30(A)(1) TABLE NO 12, FSI & BUILT\n4.\nUP AREA COMPUTATION\nArea & Road Zonal Additional Admissible Permiss\nZone width (Basic) FSI TDR ible FSI\non payment (3+4+5)\nof\nPremium\n6\n1 2 3 4 5\n--- ---\nLess than 9m 1.33 1.33\nIsland\nCity 9m & above but 1.33 0.5 0.17 2.0\nResidentia less than\nl 12.00m\n/Commer\n12.00 m 1.33 0.62 0.45 2.4\ncial\n&above but less\nthan 18.00m\n18.00m & 1.33 0.73 0.64 2.7\nabove but less\narchitects, project management consultant, valuers\nthan 27m\n27 m and above 1.33 0.84 0.83 3.0\n', '5.\nACCESS ROAD\n1. The permissible FSI on a plot is related with the road width it\nis facing as shown in FSI table.\n2. TDR and Premium FSI are not permissible on a plot fronting\nroad width less than 9 mts.\n3. MCGM shall convert all roads of width less than 9 m to 9 m\nand above as per site condition through MR&TP Act or MMC\nAct.\n4. The plots which are facing on existing roads less than 6 mt.\nwidth but are proposed to be widened upto 9 mt. in fut

In [30]:
print(sparse_results)

['1\n6 33(7) FOR OF CESSED BUILDINGS IN THE\nB.\nISLAND CITY\n1. Reconstruction or redevelopment of cessed buildings in the island\ncity by co-operative housing societies or of old buildings belonging\nto the corporation existing from prior to 30/06/1969\n2. 3 FSI on gross plot OR Rehab BUA+ Incentive whichever is more.\n3. Incentive on rehab area is 50% for 1 plot , 60% for 2 to 5 plots and\n70% 6 plots or more. An Additional incentive is permissible for\nrehab depending upon the combination of no. of plots.\n4. Each occupant shall be given the existing carpet area subject to\nminimum of 300 Sq.ft. and maximum of 1292 Sq.ft. In case of non-\nresidential occupier carpet to be given shall be equivalent to the\narea occupied in the old building.\narchitects, project management consultant, valuers\n', '1\n1\nREGULATION 30(A)(1) TABLE NO 12, FSI & BUILT\n4.\nUP AREA COMPUTATION\nArea & Road Zonal Additional Admissible Permiss\nZone width (Basic) FSI TDR ible FSI\non payment (3+4+5)\nof\nPr

In [31]:
print(hybrid_results)

['1\n1\nREGULATION 30(A)(1) TABLE NO 12, FSI & BUILT\n4.\nUP AREA COMPUTATION\nArea & Road Zonal Additional Admissible Permiss\nZone width (Basic) FSI TDR ible FSI\non payment (3+4+5)\nof\nPremium\n6\n1 2 3 4 5\n--- ---\nLess than 9m 1.33 1.33\nIsland\nCity 9m & above but 1.33 0.5 0.17 2.0\nResidentia less than\nl 12.00m\n/Commer\n12.00 m 1.33 0.62 0.45 2.4\ncial\n&above but less\nthan 18.00m\n18.00m & 1.33 0.73 0.64 2.7\nabove but less\narchitects, project management consultant, valuers\nthan 27m\n27 m and above 1.33 0.84 0.83 3.0\n', '1\n2 Area & Road Zonal Additional Admissib Permissi\nZone width (Basic) FSI le ble FSI\non payment TDR\nof (3+4+5)\nPremium\n6\n1 2 3 4 5\n--- ---\nSuburbs Less than 9m 1.00 1.00\nand\nExtended\n9m & above but 1.00 0.5 0.5 2.0\nSuburbs\nless than\n( Except\n12.00m\nBARC,\n12.00 m 1.00 0.5 0.70 2.2\nAkse,\n&above but less\nMarve and\nthan 18.00m\nCRZ)\n18.00m & 1.00 0.5 0.90 2.4\nabove but less\nthan 27m\n27 m and above 1.00 0.5 1.00 2.5\narchitects, pro

In [32]:
def doc_text_formatting(ef, query, docs):
    tokenizer = ef.model.tokenizer
    query_tokens_ids = tokenizer.encode(query, return_offsets_mapping=True)
    query_tokens = tokenizer.convert_ids_to_tokens(query_tokens_ids)
    formatted_texts = []

    for doc in docs:
        ldx = 0
        landmarks = []
        encoding = tokenizer.encode_plus(doc, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])[1:-1]
        offsets = encoding["offset_mapping"][1:-1]
        for token, (start, end) in zip(tokens, offsets):
            if token in query_tokens:
                if len(landmarks) != 0 and start == landmarks[-1]:
                    landmarks[-1] = end
                else:
                    landmarks.append(start)
                    landmarks.append(end)
        close = False
        formatted_text = ""
        for i, c in enumerate(doc):
            if ldx == len(landmarks):
                pass
            elif i == landmarks[ldx]:
                if close:
                    formatted_text += "</span>"
                else:
                    formatted_text += "<span style='color:red'>"
                close = not close
                ldx = ldx + 1
            formatted_text += c
        if close is True:
            formatted_text += "</span>"
        formatted_texts.append(formatted_text)
    return formatted_texts

In [33]:
from IPython.display import Markdown, display

# Dense search results
display(Markdown("**Dense Search Results:**"))
formatted_results = doc_text_formatting(ef, query, dense_results)
for result in dense_results:
    display(Markdown(result))

**Dense Search Results:**

1
1
REGULATION 30(A)(1) TABLE NO 12, FSI & BUILT
4.
UP AREA COMPUTATION
Area & Road Zonal Additional Admissible Permiss
Zone width (Basic) FSI TDR ible FSI
on payment (3+4+5)
of
Premium
6
1 2 3 4 5
--- ---
Less than 9m 1.33 1.33
Island
City 9m & above but 1.33 0.5 0.17 2.0
Residentia less than
l 12.00m
/Commer
12.00 m 1.33 0.62 0.45 2.4
cial
&above but less
than 18.00m
18.00m & 1.33 0.73 0.64 2.7
above but less
architects, project management consultant, valuers
than 27m
27 m and above 1.33 0.84 0.83 3.0


5.
ACCESS ROAD
1. The permissible FSI on a plot is related with the road width it
is facing as shown in FSI table.
2. TDR and Premium FSI are not permissible on a plot fronting
road width less than 9 mts.
3. MCGM shall convert all roads of width less than 9 m to 9 m
and above as per site condition through MR&TP Act or MMC
Act.
4. The plots which are facing on existing roads less than 6 mt.
width but are proposed to be widened upto 9 mt. in future, will
be permissible for FSI per 9 mt road width.
10
architects, project management consultant, valuers


1.
FLOOR SPACE INDEX (FSI)
a) FSI means the quotient of the ratio of the total
Permissible Built Up Area to the total plot area.
Permissible Built Up Area
Floor space index =
Plot area
OR
Floor space index X Plot area = Permissible Built Up Area.
e.g.
1.00 X 5000 Sq.Ft. = 5000 Sq.Ft.
5
architects, project management consultant, valuers


1
2 Area & Road Zonal Additional Admissib Permissi
Zone width (Basic) FSI le ble FSI
on payment TDR
of (3+4+5)
Premium
6
1 2 3 4 5
--- ---
Suburbs Less than 9m 1.00 1.00
and
Extended
9m & above but 1.00 0.5 0.5 2.0
Suburbs
less than
( Except
12.00m
BARC,
12.00 m 1.00 0.5 0.70 2.2
Akse,
&above but less
Marve and
than 18.00m
CRZ)
18.00m & 1.00 0.5 0.90 2.4
above but less
than 27m
27 m and above 1.00 0.5 1.00 2.5
architects, project management consultant, valuers


1
4
33(5) FOR MAHARASHTRA HOUSING & AREA
A.
DEVELOPMENT AUTHORITY (MHADA)
1. Applicable to development / redevelopment of housing schemes
implemented by MHADA on MHADA lands for Economically
Weaker Sections (EWS), Low Income Group (LIG) and Middle
Income Group (MIG) categories.
2. 3 FSI permissible on gross plot area.
3. 4 FSI is permitted to Plot area more than 4000 Sq.mt. and
fronting road width more than 18 mts.
4. Each occupant shall be given the existing carpet area + 35%
basic entitlement subject to minimum 35 Sq.mt. (376Sq.ft.)
architects, project management consultant, valuers


In [34]:
# Sparse search results
display(Markdown("\n**Sparse Search Results:**"))
formatted_results = doc_text_formatting(ef, query, sparse_results)
for result in formatted_results:
    display(Markdown(result))


**Sparse Search Results:**

1
6 33(7) FOR OF CESSED BUILDINGS IN THE
B.
ISLAND CITY
1. Reconstruction or redevelop<span style='color:red'>ment of</span> cessed<span style='color:red'> building</span>s in<span style='color:red'> the</span> island
city by co-operative housing societies or<span style='color:red'> of</span> old<span style='color:red'> building</span>s belonging
to<span style='color:red'> the</span> corporation existing from prior to 30/06/1969
2. 3<span style='color:red'> FSI</span> on gross plot OR Rehab BUA+ Incentive whichever<span style='color:red'> is</span> more.
3. Incentive on rehab area<span style='color:red'> is</span> 50%<span style='color:red'> for</span> 1 plot , 60%<span style='color:red'> for</span> 2 to 5 plots and
70% 6 plots or more. An Additional incentive<span style='color:red'> is</span> permissible<span style='color:red'> for</span>
rehab depending upon<span style='color:red'> the</span> combination<span style='color:red'> of</span> no.<span style='color:red'> of</span> plots.
4. Each occupant shall be given<span style='color:red'> the</span> existing carpet area subject to<span style='color:red'>
minimum of</span> 300 Sq.ft. and maximum<span style='color:red'> of</span> 1292 Sq.ft. In case<span style='color:red'> of</span> non-
residential occupier carpet to be given shall be equivalent to<span style='color:red'> the</span>
area occupied in<span style='color:red'> the</span> old<span style='color:red'> building</span>.
architects, project management consultant, valuers


1
1
REGULATION 30(A)(1) TABLE NO 12,<span style='color:red'> FSI</span> & BUILT
4.
UP AREA COMPUTATION
Area & Road Zonal Additional Admissible Permiss
Zone width (Basic)<span style='color:red'> FSI</span> TDR ible<span style='color:red'> FSI</span>
on payment (3+4+5)<span style='color:red'>
of</span>
Premium
6
1 2 3 4 5
--- ---
Less than 9<span style='color:red'>m</span> 1.33 1.33
Island
City 9<span style='color:red'>m</span> &<span style='color:red'> above</span> but 1.33 0.5 0.17 2.0
Residentia less than
l 12.00<span style='color:red'>m</span>
/Commer
12.00 m 1.33 0.62 0.45 2.4
cial
&above but less
than 18.00<span style='color:red'>m</span>
18.00<span style='color:red'>m</span> & 1.33 0.73 0.64 2.7<span style='color:red'>
above</span> but less
architects, project management consultant, valuers
than 27<span style='color:red'>m</span>
27 m and<span style='color:red'> above</span> 1.33 0.84 0.83 3.0


1
7
C. 33(7)(A) FOR TENANT OCCUPIED BUILDING
1. Applicable to Dilapidated authorized tenanted<span style='color:red'> building</span>/s in Suburbs
and Dilapidated authorized Non cessed Tenanted<span style='color:red'> building</span>/s in Island
City
2. Rehab area + Incentive OR Permissible<span style='color:red'> FSI</span> as per Table no 12<span style='color:red'> of</span>
regulation 30(A)(1) whichever<span style='color:red'> is</span> more.
3. Incentive on rehab area<span style='color:red'> is</span> 50%<span style='color:red'> for</span> 1 plot , 60%<span style='color:red'> for</span> 2 to 5 plots and
70%<span style='color:red'> for</span> 6 plots or more.
4. Each tenant will get<span style='color:red'> minimum</span> 300 Sq.ft. and maximum<span style='color:red'> of</span> 1292 Sq.ft.<span style='color:red'>
of</span> carpet area and non-residential occupier will get equivalent to
existing carpet area.
5. No new tenancy created after 13/6/96 shall be considered.
architects, project management consultant, valuers


1
2 Area & Road Zonal Additional Admissib Permissi
Zone width (Basic)<span style='color:red'> FSI</span> le ble<span style='color:red'> FSI</span>
on payment TDR<span style='color:red'>
of</span> (3+4+5)
Premium
6
1 2 3 4 5
--- ---
Suburbs Less than 9<span style='color:red'>m</span> 1.00 1.00
and
Extended
9<span style='color:red'>m</span> &<span style='color:red'> above</span> but 1.00 0.5 0.5 2.0
Suburbs
less than
( Except
12.00<span style='color:red'>m</span>
BARC,
12.00 m 1.00 0.5 0.70 2.2
Akse,
&above but less
Marve and
than 18.00<span style='color:red'>m</span>
CRZ)
18.00<span style='color:red'>m</span> & 1.00 0.5 0.90 2.4<span style='color:red'>
above</span> but less
than 27<span style='color:red'>m</span>
27 m and<span style='color:red'> above</span> 1.00 0.5 1.00 2.5
architects, project management consultant, valuers


1
9
33(9) FORBUILDINGS UNDER CLUSTER
E.
DEVELOPMENT SCHEME(S)(CDS)
1. For, Redevelop<span style='color:red'>ment of</span> Clusters. Minimum Plot area<span style='color:red'> of</span> 4000
Sq.<span style='color:red'>m</span>t. in Island City and 6000 Sq.<span style='color:red'>m</span>t. in Suburbs. Road width must
be 18 mt. wide or more.
2. Cessed Buildings, Non cessed<span style='color:red'> building</span>s which are 30 years old,
Slums upto 50%<span style='color:red'> of</span> area are allowable under Cluster
redevelop<span style='color:red'>ment</span>.
2. 4<span style='color:red'> FSI</span> OR Rehab Built up area + Incentive whichever<span style='color:red'> is</span> more.
3. Each rehab tenement shall be given occupied area subject to<span style='color:red'>
minimum</span> 300 Sq.ft. carpet area and maximum<span style='color:red'> of</span> 1076 Sq.ft.(100
Sq.<span style='color:red'>m</span>t.)
architects, project management consultant, valuers


In [35]:
# Hybrid search results
display(Markdown("\n**Hybrid Search Results:**"))
formatted_results = doc_text_formatting(ef, query, hybrid_results)
for result in formatted_results:
    display(Markdown(result))


**Hybrid Search Results:**

1
1
REGULATION 30(A)(1) TABLE NO 12,<span style='color:red'> FSI</span> & BUILT
4.
UP AREA COMPUTATION
Area & Road Zonal Additional Admissible Permiss
Zone width (Basic)<span style='color:red'> FSI</span> TDR ible<span style='color:red'> FSI</span>
on payment (3+4+5)<span style='color:red'>
of</span>
Premium
6
1 2 3 4 5
--- ---
Less than 9<span style='color:red'>m</span> 1.33 1.33
Island
City 9<span style='color:red'>m</span> &<span style='color:red'> above</span> but 1.33 0.5 0.17 2.0
Residentia less than
l 12.00<span style='color:red'>m</span>
/Commer
12.00 m 1.33 0.62 0.45 2.4
cial
&above but less
than 18.00<span style='color:red'>m</span>
18.00<span style='color:red'>m</span> & 1.33 0.73 0.64 2.7<span style='color:red'>
above</span> but less
architects, project management consultant, valuers
than 27<span style='color:red'>m</span>
27 m and<span style='color:red'> above</span> 1.33 0.84 0.83 3.0


1
2 Area & Road Zonal Additional Admissib Permissi
Zone width (Basic)<span style='color:red'> FSI</span> le ble<span style='color:red'> FSI</span>
on payment TDR<span style='color:red'>
of</span> (3+4+5)
Premium
6
1 2 3 4 5
--- ---
Suburbs Less than 9<span style='color:red'>m</span> 1.00 1.00
and
Extended
9<span style='color:red'>m</span> &<span style='color:red'> above</span> but 1.00 0.5 0.5 2.0
Suburbs
less than
( Except
12.00<span style='color:red'>m</span>
BARC,
12.00 m 1.00 0.5 0.70 2.2
Akse,
&above but less
Marve and
than 18.00<span style='color:red'>m</span>
CRZ)
18.00<span style='color:red'>m</span> & 1.00 0.5 0.90 2.4<span style='color:red'>
above</span> but less
than 27<span style='color:red'>m</span>
27 m and<span style='color:red'> above</span> 1.00 0.5 1.00 2.5
architects, project management consultant, valuers


5.
ACCESS ROAD
1. The permissible<span style='color:red'> FSI</span> on a plot<span style='color:red'> is</span> related with<span style='color:red'> the</span> road width it<span style='color:red'>
is</span> facing as shown in<span style='color:red'> FSI</span> table.
2. TDR and Premium<span style='color:red'> FSI</span> are not permissible on a plot fronting
road width less than 9 mts.
3. MCGM shall convert all roads<span style='color:red'> of</span> width less than 9 m to 9 m
and<span style='color:red'> above</span> as per site condition through MR&TP Act or MMC
Act.
4. The plots which are facing on existing roads less than 6 mt.
width but are proposed to be widened upto 9 mt. in future, will
be permissible<span style='color:red'> for FSI</span> per 9 mt road width.
10
architects, project management consultant, valuers


1.
FLOOR SPACE INDEX (F<span style='color:red'>SI</span>)
a)<span style='color:red'> FSI</span> means<span style='color:red'> the</span> quotient<span style='color:red'> of the</span> ratio<span style='color:red'> of the</span> total
Permissible Built Up Area to<span style='color:red'> the</span> total plot area.
Permissible Built Up Area
Floor space index =
Plot area
OR
Floor space index X Plot area = Permissible Built Up Area.
e.g.
1.00 X 5000 Sq.Ft. = 5000 Sq.Ft.
5
architects, project management consultant, valuers


1
4
33(5) FOR MAHARASHTRA HOUSING & AREA
A.
DEVELOPMENT AUTHORITY (MHADA)
1. Applicable to development / redevelop<span style='color:red'>ment of</span> housing schemes
implemented by MHADA on MHADA lands<span style='color:red'> for</span> Economically
Weaker Sections (EWS), Low Income Group (LIG) and Middle
Income Group (MIG) categories.
2. 3<span style='color:red'> FSI</span> permissible on gross plot area.
3. 4<span style='color:red'> FSI is</span> permitted to Plot area more than 4000 Sq.<span style='color:red'>m</span>t. and
fronting road width more than 18 mts.
4. Each occupant shall be given<span style='color:red'> the</span> existing carpet area + 35%
basic entitle<span style='color:red'>ment</span> subject to<span style='color:red'> minimum</span> 35 Sq.<span style='color:red'>m</span>t. (376Sq.ft.)
architects, project management consultant, valuers


In [36]:
final_template = f"""
You are an expert assistant providing detailed and accurate information. The user has asked the following question:
**Question:** {query}

Below is relevant information retrieved from various sources:
**Retrieved Information:**
{hybrid_results}

**Instruction:** Use only the query-relevant content from the retrieved information to answer the question. Focus on providing a comprehensive, informative response based solely on the given data. If any conflicting details are present, prioritize the most reliable and consistent information.
"""
final_template

"\nYou are an expert assistant providing detailed and accurate information. The user has asked the following question:\n**Question:** What is the minimum requirement of FSI for building above 70m\n\nBelow is relevant information retrieved from various sources:\n**Retrieved Information:**\n['1\\n1\\nREGULATION 30(A)(1) TABLE NO 12, FSI & BUILT\\n4.\\nUP AREA COMPUTATION\\nArea & Road Zonal Additional Admissible Permiss\\nZone width (Basic) FSI TDR ible FSI\\non payment (3+4+5)\\nof\\nPremium\\n6\\n1 2 3 4 5\\n--- ---\\nLess than 9m 1.33 1.33\\nIsland\\nCity 9m & above but 1.33 0.5 0.17 2.0\\nResidentia less than\\nl 12.00m\\n/Commer\\n12.00 m 1.33 0.62 0.45 2.4\\ncial\\n&above but less\\nthan 18.00m\\n18.00m & 1.33 0.73 0.64 2.7\\nabove but less\\narchitects, project management consultant, valuers\\nthan 27m\\n27 m and above 1.33 0.84 0.83 3.0\\n', '1\\n2 Area & Road Zonal Additional Admissib Permissi\\nZone width (Basic) FSI le ble FSI\\non payment TDR\\nof (3+4+5)\\nPremium\\n6\\n1 2 

In [37]:
from groq import Groq
client = Groq(
    api_key='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' ,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": final_template,
        }
    ],
    model="llama-3.1-70b-versatile",
)
print(chat_completion.choices[0].message.content)

Based on the provided information, I couldn't find any specific details about the minimum FSI requirement for buildings above 70 meters. However, I found some general guidelines and FSI calculations for different building heights, which might be related to the query.

For buildings above 27 meters, the FSI is as follows:

- In the Island City Residential/Commercial zone and suburbs (except BARC, Akse, Marve, and CRZ): 2.5 (composed of 1 (Basic FSI) + 0.5 (Additional FSI) + 1 (TDR))
- In the Island City Residential/Commercial zone but not specifically mentioned in other zones: 3.0

Considering these calculations, we can infer the following:

- A minimum FSI of 2.5 is applicable for buildings above 27 meters and not exceeding 70 meters in the specified zones (excluding BARC, Akse, Marve, and CRZ).
- However, this might not be a comprehensive answer, and additional regulations might be applicable for specific zones or areas.

In the absence of explicit information on the FSI requirements 